## IPFS Client
This notebook interacts with our local IPFS node (and consequently the configured IPFS private network), using our Python module, as to simulate a Jupyter Server extension (i.e. the backend).

### Python module setup

In [1]:
import sys
!{sys.executable} -m pip install *.whl

Processing ./jcipfsclient-1.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.2 MB/s eta 0:00:0000:01


In [2]:
import jcipfsclient as ipfs

### Configuration

In [3]:
nodeApiUrl = 'http://ipfs.jupyter.localhost:5001'
ipfs.getId(nodeApiUrl)

'12D3KooWJ9P1xruGf9WpXKsUv9UBPuNSnyYaeWKAg6fyvz2yCwxF'